# 3.6: Inspecting `HCLG.fst`

이전 `notebook`들을 실행시켜서 우리는 `HCLG.fst`라는 아주 큰 `FST`를 구축하였습니다. 세 번에 나누어서 실행하였기 때문에, `HCLG.fst` 파일은 `exp` 디렉토리 하위에 있는 `monophones`, `triphones`, 그리고 `triphones_lda` 디렉토리 아래에 각각 위치하고 있습니다. 

In [ ]:
%%bash
ls -lah exp/monophones/graph/HCLG.fst

In [ ]:
%%bash
ls -lah exp/triphones/graph/HCLG.fst

In [ ]:
%%bash
ls -lah exp/triphones_lda/graph/HCLG.fst


파일 크기를 비교해보면 layer가 높아질 수록 `HCLG`의 크기도 커지는 것을 알 수 있습니다. 

`HCLG`에 대해서 더 자세히 살펴보기 전 [이 포스트](http://vpanayotov.blogspot.com/2012/06/kaldi-decoding-graph-construction.html)를 읽어보면  `HCLG`에 대해 더 잘 이해할 수 있을 것입니다. 

또한 [이 튜토리얼](https://github.com/michaelcapizzi/kaldi/blob/kaldi_instructional/egs/INSTRUCTIONAL/resource_files/resources/wfst_tutorial.pdf)의 **Section 6**을 통해서 `kaldi` [공식 페이퍼](https://github.com/michaelcapizzi/kaldi/blob/kaldi_instructional/egs/INSTRUCTIONAL/resource_files/resources/wfst_paper.pdf)에서 말하는 `HCLG`의 기본 개념에 대해 알아볼 수 있습니다. 


## Building `HCLG`

`HCLG`는 우리가 구축한 `ASR` pipeline의 **모든** 내용을 담고 있는 하나의 `FST`입니다. 이 `FST`는 **네 개**의 `FST`를 합쳐놓은 것이라고 볼 수 있습니다. 

`Decoding`은 두 단계로 이루어집니다. 
 1. 어떤 `GMM`이 주어진 프레임과 가장 일치하는지
 2. 어떤 경로로 가는 것이 가장 효율적일지



`HCLG`를 살펴보기 위해서, 이전에 사용했던 패키지와 같은 것을 사용하겠습니다. 

In [ ]:
# because of the way `kaldi` installed `openFST` we have to add the path to the python functions here
import sys
sys.path.append("/scratch/kaldi/tools/openfst-1.6.2/lib/python2.7/site-packages")    

from utils.fst_manipulate import fst_manipulate as fstman  # scripts to further manipulate fsts

import pywrapfst as openfst  # the wrapper module
import graphviz as dot       # a wrapper for graphviz, which will allow us to visualize

### $G$

`G`는 language model을 `FST`로 표현한 것입니다. 이전 노트북에서 자세한 내용은 확인할 수 있습니다. 이 `FST`에는 `단어:단어` 관계를 담고 있으며, 아래는 toy corpus에서 구축한 `language model`을 나타낸 것입니다. 

In [ ]:
G_animal = openfst.Fst.read("resource_files/fst/animal_fst-2_gram.fst")
G_animal

실제 `G.fst` 파일은 `data/lang_test_tg`에서 찾을 수 있습니다. 이 파일은 훨씬 이전에 `run_prepare_data.sh` 파일을 실행하였을 때 만들어진 파일입니다. 

**Note:** 실제 `FST`는 너무 크기 때문에 시각화하기는 어렵습니다. 하지만 그 중 일부 정보는 확인해볼 수 있습니다. 

In [ ]:
%%bash
ls -lah data/lang_test_tg/G.fst

In [ ]:
G = openfst.Fst.read("data/lang_test_tg/G.fst")

우리는 실제 `G.fst`에 몇 개의 state가 있는지 `.states()`에 있는 `iterator`를 `<list>`로 변환하고, 그 list의 길이를 구함으로써 확인해볼 수 있습니다. 

In [ ]:
G_states = len(list(G.states()))
G_states

이와 비슷한 방법으로 `.arcs()`를 통해서 해당 state가 몇 개의 `arc`를 가지고 있는지 확인할 수 있습니다. (`.arcs()`는 `state` 하나를 논항으로 갖습니다.)

In [ ]:
G_arcs = 0
for s in G.states():
    intermediate_arcs = len(list(G.arcs(s)))
    G_arcs += intermediate_arcs
G_arcs

아래는 위의 두 가지를 각각 함수로 구현해둔 것입니다. 

In [ ]:
def get_num_states(fst_in):
    return len(list(fst_in.states()))

def get_num_arcs(fst_in):
    num_arcs = 0
    for s in fst_in.states():
        intermediate_arcs = len(list(fst_in.arcs(s)))
        num_arcs += intermediate_arcs
    return num_arcs

우리는 `arc`가 가지고 있는 정보 또한 읽어올 수 있습니다. 

**Note:** 우리는 처음 **다섯 개**의 state만 확인하도록 하겠습니다. 그를 위해 `for` 루프에서 다섯 개를 확인한 이후, `break` 명령어를 실행하겠습니다. 

In [ ]:
c = 0
for s in G.states():
    for a in G.arcs(s):
        if c < 5:
            print("in: {}\nout: {}\nweight: {}".format(
                a.ilabel,
                a.olabel,
                a.weight
                )
            )
            print("=========")
        else:
            break
        c += 1
    break

`openfst`에서는 `arc`에서 실제 문자열 대신에 `index`를 사용한다는 것을 기억하세요. 그렇기 때문에 `kaldi`에서 제공하는 "lookup" 기능을 이용하여 `index`를 실제 문자로 바꿔줄 필요가 있습니다. 

In [ ]:
%%bash
cat data/lang/words.txt | grep -E " 2$"       # the -E flag will allow us to use a regex
cat data/lang/words.txt | grep -E " 49$"
cat data/lang/words.txt | grep -E " 220$"
cat data/lang/words.txt | grep -E " 227$"
cat data/lang/words.txt | grep -E " 456$"

여기에서 우리는 `ilable`과 `olabel`이 같다는 것을 확인할 수 있습니다. 이를 통해서 우리는 `arc`가 `단어:단어`의 구조를 가지며, `weight`는 probability를 나타낸다는 것을 확인할 수 있습니다. 

### $L$

`L`은 `lexicon`을 나타내는 `FST`입니다. 이것은 `Inspecting data dir`을 진행하면서 이미 `data` 디렉토리 아래에 생성해 둔 것입니다. 

In [ ]:
ls -lah data/lang/ | grep L

In [ ]:
L = openfst.Fst.read("data/lang/L.fst")
print("number of states in L: {}".format(get_num_states(L)))
print("number of arcs in L: {}".format(get_num_arcs(L)))

In [ ]:
c = 0
for s in L.states():
    for a in L.arcs(s):
        if c < 15:
            print("in: {}\nout: {}\nweight: {}".format(
                a.ilabel,
                a.olabel,
                a.weight
                )
            )
            print("=========")
        else:
            break
        c += 1
    if c > 15:
        break

`L`은 `arc` 구조가 조금 다릅니다. `L`에서는 `단어:단어` 대신에 `phone:단어`의 구조를 가지며, 그렇기 때문에 우리는 `phone`의 index를 변환해주는 "lookup" 기능도 사용하여야 합니다. 

In [ ]:
%%bash
cat data/lang/phones.txt | grep -E " 122$"
cat data/lang/words.txt | grep -E " 8$"

여기에서 `weight`가 나타내는 것은 명확하지 않습니다. 하지만 들여다보면 그 값들은 거의 항상 **동일**하다는 것을 알 수 있습니다. (또는 0.) 그렇기 때문에 모든 `arc`의 `weight`가 동일하다면 가장 효율적인 경로를 찾을 필요가 없어집니다. 

**Note:** 단어가 하나 이상의 phone으로 구성되었을 경우 어떻게 처리하는지 궁금할 수 있습니다. 그 경우, **제일 처음** `arc`는 `phone:단어`의 구조를 가지며 이어지는 `arc`는 `phone:<eps>`의 구조를 가집니다. 여기서 `<eps>`는 $epsilon$을 나타냅니다. `L`을 시각화한 예제를 [이 곳](http://vpanayotov.blogspot.com/2012/06/kaldi-decoding-graph-construction.html)에서 확인할 수 있습니다. 

### $LG = L \circ G$

`L`과 `G`를 합치게 되면, 우리는 **철자**와 **language model** 정보를 하나로 합친 결과를 얻게 됩니다. 최종적으로 얻은 `FST`는 `phone:단어`의 구조를 가진 `arc`로 구성될 것입니다. 

**Note:** 일반적으로 두 개를 합칠 경우, 최종 `FST`의 `arc`는 첫 번째 `FST`의 **`input`** 을 왼쪽에 갖고, 오른쪽에는 두 번째 `FST`의 **`output`** 을 갖게 됩니다.

**Note:** 만약 이전에 `temp` 디렉토리를 지우지 않았을 경우, `LG.fst` 파일이 남아있게 됩니다. 

In [ ]:
%%bash
ls -lah data/lang_test_tg/tmp/LG.fst

`LG.fst` 파일이 `L`이나 `G`보다 큰 것은 당연합니다. 

In [ ]:
LG = openfst.Fst.read("data/lang_test_tg/tmp/LG.fst")

In [ ]:
c = 0
for s in LG.states():
    for a in LG.arcs(s):
        if c < 10:
            print("in: {}\nout: {}\nweight: {}".format(
                a.ilabel,
                a.olabel,
                a.weight
                )
            )
            print("=========")
        else:
            break
        c += 1
    if c > 10:
        break

In [ ]:
%%bash
cat data/lang/phones.txt | grep -E " 14$"
cat data/lang/words.txt | grep -E " 79970$"

그리고 $G$의 `arc weights`가 language model의 probability를 나타내기 때문에, 그 값들은 $LG$로 전파됩니다. 

### $C$

`C`는 acoustic model의 `triphone` 정보를 담고 있습니다. 

**Note:** 이 `FST`는 `kaldi`에서 독립된 모델로 만들어지지는 않습니다. 그렇기 때문에, 우리가 `G`나 `L`을 살펴본 것과 같은 방식으로 살펴볼 수는 없습니다. 하지만 [이 포스트](http://vpanayotov.blogspot.com/2012/06/kaldi-decoding-graph-construction.html)는 전체적으로 $C$가 어떤 구조인지를 보여줍니다. 또한 포스트를 통해서 `indexing`의 개념을 이해할 수 있습니다. 

 ```
 The input symbols of the C graph are triphone IDs, which are specified by using a Kaldi-specific data structure called ilabel_info(frankly clabel_info would have been more intuitive name for me, but perhaps there is reason it's called that way). Basically this is an array of arrays, where the the indices of the first dimension are the triphone IDs and the individual entries of the nested arrays are the IDs of the context-independent phones, which constitute the context window for the particular triphone. For example if there are triphone "a/b/c"(i.e. central phone "b" with left context "a" and right context "c") with id "10" the eleventh entry in the ilabel_info will be an array containing the context-independent ID's of the phones "a", "b" and "c".```

결론적으로, `arc`는 `triphone:phone`의 형태를 담고 있으며, 여기에서 `phone`은 `ARPAbet phone`으로 각각의 단어를 나타내기 위해 사용된 phone입니다. 

In [ ]:
%%bash
head data/lang/phones.txt

### $CLG = C \circ LG$

`C (triphone:phone)`과 `LG (phone:word)`를 합친 것이기 때문에, 최종 `FST`의 `arc`는 `triphone:word`의 정보를 담게 됩니다. 그리고 여전히 `arc`의 `weight`는 language model probability입니다. 

### $H$

Acoustic model은 `HMM`을 이용해서 만들어집니다. 그러므로 `H`는 `HMM` 정보를 graph에 합치는 과정입니다. `HMM`의 `arc`는 `transition_id:triphone`의 구조를 갖고 있습니다. `transition_id`는 네 개로 나누어볼 수 있습니다. 
 1. `phone`
 2. `HMM state`
 3. `PDF`
 4. `HMM`에서 나가는 `arc`
 
[이 포스트](http://vpanayotov.blogspot.com/2012/06/kaldi-decoding-graph-construction.html)를 통해서 예시를 확인할 수 있습니다. 

```
For example "k_1_739_1" in this notation means that this is the transition-id associated with the state "1"(i.e. the second state) of the monophone "k" having PDF ID of 739(this is in general different for the different context-dependent versions of "k") and the outgoing arc from this HMM state, which has ID "1". 
```

### $HCLG = H \circ CLG$

최종 graph는 네 개의 파트를 합친 것입니다. 그리고 가장 중요한 `arc`의 구조는 `transition_id:word`와 같습니다. 이 정보가 바로 새롭게 주어진 음성 파일이 어떤 내용인지 예측하기 위해서 필요한 정보입니다. 어떤 프레임이 주어지더라도 `GMM`은 어떤 `transition_id`를 사용할지 선택하게 되고, 각각의 프레임에서 해당 정보를 추출하여 결국 연속된 프레임이 어떠한 `단어`를 나타내는지 graph를 통해서 예측할 수 있습니다. 

**Note:** `HaCLG.fst` 파일과 `HCLG.fst` 파일 두 개가 존재하는 것을 확인할 수 있습니다. `HaCLG.fst` 파일은 단순한 intermediate 과정에서 필요한 파일입니다. 자세한 내용이 궁금하시면 [이 곳](http://vpanayotov.blogspot.com/2012/06/kaldi-decoding-graph-construction.html)에서 확인하실 수 있습니다. 우리는 최종 모델만 살펴보겠습니다. 

## Comparing `HCLG` from different acoustic models

우리는 **세 개**의 다른 `HCLG.fst` 파일을 생성하였습니다. 
 1. `monophone` acoustic model
 2. `triphone` acoustic model
 3. `triphone_lda` acoustic model

각각의 파일 크기를 살펴보면 서로 다르다는 것을 볼 수 있습니다. 

In [ ]:
%%bash
ls -lah exp/monophones/graph/HCLG.fst
ls -lah exp/triphones/graph/HCLG.fst
ls -lah exp/triphones_lda/graph/HCLG.fst

In [ ]:
HCLG_mono = openfst.Fst.read("exp/monophones/graph/HCLG.fst")
HCLG_tri = openfst.Fst.read("exp/triphones/graph/HCLG.fst")
HCLG_tri_lda = openfst.Fst.read("exp/triphones_lda/graph/HCLG.fst")

print("number of states compared\nmono: {}\ntri: {}\ntri_lda: {}\n\n".format(
        get_num_states(HCLG_mono),
        get_num_states(HCLG_tri),
        get_num_states(HCLG_tri_lda)
    )
)
print("====================")
print("number of arcs compared\nmono: {}\ntri: {}\ntri_lda: {}".format(
        get_num_arcs(HCLG_mono),
        get_num_arcs(HCLG_tri),
        get_num_arcs(HCLG_tri_lda)
    )
)

우리는 **세 개**의 `HCLG`를 구축하면서 **완벽하게 동일한** `L`과 `G`를 사용하였습니다. 그러므로 차이를 만들어 낸 것은 `C`와 `H`일 것입니다. 

이전 내용을 기억하신다면, `triphone` 모델에서 실제로는 `monophone` 모델보다 **더 작은** 크기의 `decision tree`를 사용했다는 것을 아실 수 있습니다. 그 이유는 `clustering`이 어느 정도 이루어졌기 때문입니다. 그리고 `triphone_lda` 모델은 `num_leaves`의 크기가 컸기 때문에 `decision tree` 자체도 `triphone` 모델보다 컸다는 것을 알고 계실 것입니다. `num_leaves` 값이 `H`의 크기에 영향을 끼치기 때문에, `HCLG`에서 state의 갯수에도 영향을 끼치게 됩니다. 

파일의 크기에 따라서 새로운 음성 파일을 decoding하는데 걸리는 시간이 달라집니다. 이어지는 `notebook`을 실행해보시면 그 차이를 아실 수 있습니다. 